todo:
- find the outliers, where the deposit does not equal the payment, all the others can be classified
- asign the transactions to the dkb account so that we can filter them out
    

In [1]:
import sys, os


import logging
import pandas as pd
import numpy as np
from pathlib import Path
# sys.path.insert(0,'..')
project_dir = str(Path(os.getcwd()).parent)
if not project_dir in sys.path:
    sys.path.insert(0,project_dir)
sys.path
from BankClassify import BankClassify, assign_target_account, assign_label
from load_data import read_dkb_csv, read_paypal_csv
from classify_helper import merge_paypal_enties, check_balance
import yaml

In [2]:
%load_ext autoreload

%autoreload 2

In [3]:
logging.basicConfig(level=logging.DEBUG)

In [4]:
input_data_path =  Path('/Users/jangie/Documents/Finanzen/Detailed_Data')

In [5]:
# bc.data['dkb_3109'].index = len(bc.data['dkb_3109'])

In [6]:
bc = BankClassify(Path('../data'))

DEBUG:root:FFFFF /Users/jangie/Projects/DataScience/BankClassify/BankClassify.py
INFO:root:loaded previous dkb_3109 data 541 entries
INFO:root:loaded previous paypal data 1017 entries
INFO:root:loaded previous dkb_4217 data 1212 entries
INFO:root:train dataset size 2145


In [7]:
bc.data_all.index.levels[0]

Index(['dkb_3109', 'paypal', 'dkb_4217'], dtype='object')

In [11]:
bc.data_all

datetime   amount  \
dkb_3109 0     2021-12-29 00:00:00   -39.99   
         1     2021-12-27 00:00:00   -57.00   
         2     2021-12-27 00:00:00   -16.79   
         3     2021-12-23 00:00:00  3845.20   
         4     2021-12-22 00:00:00   500.00   
...                            ...      ...   
dkb_4217 1203  2022-01-21 00:00:00   -17.25   
         1204  2022-01-21 00:00:00   -21.95   
         1205  2022-01-20 00:00:00   -40.96   
         1206  2022-01-20 00:00:00    -8.22   
         1211  2022-01-07 00:00:00   -19.50   

                                                            desc  \
dkb_3109 0     Lastschrift Vodafone Deutschland GmbH 12/2021 ...   
         1     Überweisung EDITH ESPINOZA DE THIEKE Rechn.Nr....   
         2     Lastschrift Vodafone GmbH 0000114111198 001508...   
         3     Lohn, Gehalt, Rente IAV GmbH Ingenieurgesellsc...   
         4     Gutschrift DR.ST. U. C. GIESELER Feliz natal p...   
...                                                          ...   
dkb_4217 1203  Kartenzahlung/-abrechnung DM FIL.2814 H:65273/...   
         1204  Kartenzahlung/-abrechnung HUGENDUBEL 0665 FIL....   
         1205  Kartenzahlung/-abrechnung EDEKA ZAHL, BERLIN//...   
         1206  Kartenzahlung/-abrechnung HANSEN OBST GBR GIR ...   
         1211  Lastschrift SWAPFIETS BY BUCKAROO Swapfiets: D...   

              target account                               class  
dkb_3109 0               NaN               Bill - Communications  
         1               NaN       Bill - Utilities, Rent & Fees  
         2               NaN               Bill - Communications  
         3               NaN          Income - Salary & Benefits  
         4               NaN                   Income - Presents  
...                      ...                                 ...  
dkb_4217 1203            NaN  Supermarket & Everyday commodities  
         1204            NaN                               Other  
         1205            NaN  Supermarket & Everyday commodities  
         1206            NaN  Supermarket & Everyday commodities  
         1211            NaN                    Public Transport  

[1856 rows x 5 columns]

In [8]:
for f in Path('../data').glob('*.csv'):
    print(f.with_suffix('').name)

dkb_3109
dkb_4217


In [9]:
# bc.data

In [10]:
sorted([f for f in (input_data_path/'dkb').glob('*.csv')])

[PosixPath('/Users/jangie/Documents/Finanzen/Detailed_Data/dkb/1002423109.csv'),
 PosixPath('/Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2020_1002423109.csv'),
 PosixPath('/Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2020_1071944217.csv'),
 PosixPath('/Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2021_1002423109.csv'),
 PosixPath('/Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2021_1071944217.csv'),
 PosixPath('/Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2022_1002423109.csv'),
 PosixPath('/Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2022_1071944217.csv')]

### load new raw data and add to existing dataset

#### paypal data

In [1]:
filename = Path('/Users/jangie/Documents/Finanzen/Detailed_Data/Paypal/Paypal_01 Jan 2019 - 29 Dec 2022.CSV')
df_pp = read_paypal_csv(filename)
print(f"paypal datasize is {len(df_pp)}")
merge_paypal_enties(df_pp)
print(f"after merge reduced to {len(df_pp)}")
check_balance(df_pp)

# consitency check, merge_paypal_enties, should eliminate all the foreig currency entries
assert (df_pp['Currency'].unique() == ['EUR']).all()

# enties with identical datetime, there should be one that is the transfer to the dkb bank
# if we eliminate these, the remaining datetimes should be unique
assert (df_pp[df_pp['target account'].isna()]['datetime'].value_counts().unique() == [1]).all()

NameError: name 'Path' is not defined

In [39]:
# df_pp[['datetime', 'amount', 'desc', 'target account', 'class']]

In [40]:
bc.add_data(df_pp, "paypal")

DEBUG:root:adding paypal data!
DEBUG:root:previous dataset 1017 entries
DEBUG:root:new dataset 1017 entries with 0 new ones


In [14]:
# bc.data.keys()

In [15]:
# pd.concat(bc.data.values(), keys=bc.data.keys())[['datetime', 'amount', 'desc']]

In [22]:
# bc.data['paypal']

In [25]:
bc.data_unlabeled

datetime  amount  \
paypal 0     2019-05-27 07:02:28+00:00 -100.00   
       1     2020-09-03 08:31:34+00:00    6.00   
       2     2020-09-26 06:53:18+00:00  -98.22   
       4     2020-10-06 03:05:58+00:00  -10.98   
       6     2020-10-09 17:11:33+00:00  -32.70   
...                                ...     ...   
       1007  2022-12-08 18:36:59+00:00   -9.40   
       1009  2022-12-09 00:21:18+00:00  -59.99   
       1011  2022-12-09 10:54:04+00:00  -65.00   
       1013  2022-12-10 18:24:47+00:00  -34.15   
       1015  2022-12-16 12:49:28+00:00  -12.99   

                                                          desc target account  \
paypal 0     Alexander Kihm   Alles Liebe und viel Spass!! ...            NaN   
       1                                     Konrad Exner   �             NaN   
       2     DURSTEXPRESS GmbH Germany Bestellung 10585-247...            NaN   
       4     baby-walz GmbH Germany paypal-payment:62133221...            NaN   
       6                        Takeaway.com Payments B.V.                NaN   
...                                                        ...            ...   
       1007  LogPay Financial Services GmbH  BVG Tickets 4-...            NaN   
       1009                Fashion ID GmbH & Co. KG Germany               NaN   
       1011  CTS EVENTIM Sports GmbH  Tickets Alba Berlin  ...            NaN   
       1013  app smart GmbH / order smart Germany Salmon Lo...            NaN   
       1015                                     Spotify AB                NaN   

            class  
paypal 0      NaN  
       1      NaN  
       2      NaN  
       4      NaN  
       6      NaN  
...           ...  
       1007   NaN  
       1009   NaN  
       1011   NaN  
       1013   NaN  
       1015   NaN  

[532 rows x 5 columns]

#### dkb data

In [43]:
filename = Path('/Users/jangie/Documents/Finanzen/Detailed_Data/dkb/1002423109.csv')

In [44]:
df_new = read_dkb_csv(filename)

INFO:root:number of data loaded: 70
DEBUG:root:new data from /Users/jangie/Documents/Finanzen/Detailed_Data/dkb/1002423109.csv with 70 entries, years [2022]


In [46]:
df_new['date']

0    2022-09-30
1    2022-09-30
2    2022-09-29
3    2022-09-28
4    2022-09-27
        ...    
65   2022-08-04
66   2022-08-03
67   2022-08-03
68   2022-08-02
69   2022-08-01
Name: date, Length: 70, dtype: datetime64[ns]

In [11]:
len(bc.data['dkb_3109'][bc.data['dkb_3109']['class'].isna()])

123

In [12]:
for k, v in bc.data.items():
    print(f"{k}: {len(v)} entires")

dkb_3109: 541 entires
dkb_4217: 1212 entires


In [13]:
df_new

,date,amount,desc,account_nr
0,2022-09-30,-315.64,Kartenzahlung/-abrechnung VIVA RESORT PROT.PRI...,1002423109
1,2022-09-30,-4.00,Kartenzahlung/-abrechnung H.RESORT SUPERMERCAD...,1002423109
2,2022-09-29,-39.99,Lastschrift Vodafone Deutschland GmbH 09/2022 ...,1002423109
3,2022-09-28,4157.32,"Lohn, Gehalt, Rente IAV GmbH Ingenieurgesellsc...",1002423109
4,2022-09-27,-69.00,Überweisung GISELA SILVA Rechnungs-Nr.: 142/20...,1002423109
...,...,...,...,...
65,2022-08-04,-40.00,Kartenzahlung/-abrechnung RESTAURANT SONNE GIR...,1002423109
66,2022-08-03,-100.00,Lastschrift European Bank for Financial Servic...,1002423109
67,2022-08-03,-23.00,Lastschrift GIRASOLES - SONNENBLUMEN Beitrag N...,1002423109
68,2022-08-02,-7.20,Kartenzahlung/-abrechnung Lagard re Travel Ret...,1002423109


In [14]:
bc.add_data(df_new, f"dkb_{filename.with_suffix('').name[-4:]}")

DEBUG:root:adding dkb_3109 data!
DEBUG:root:previous dataset 541 entries
DEBUG:root:new dataset 70 entries with 0 new ones


In [18]:
bc.data_unlabeled

date    amount  \
dkb_3109 477 2022-09-30   -315.64   
         478 2022-09-30     -4.00   
         479 2022-09-29    -39.99   
         480 2022-09-28   4157.32   
         481 2022-09-27    -69.00   
         483 2022-09-26    -29.84   
         484 2022-09-26    -19.99   
         485 2022-09-22    -48.00   
         486 2022-09-21  -1000.00   
         487 2022-09-21  -1000.00   
         488 2022-09-20    -90.00   
         489 2022-09-19    -48.00   
         490 2022-09-19    -42.80   
         491 2022-09-19    -18.59   
         492 2022-09-19    -11.94   
         493 2022-09-19     -5.00   
         495 2022-09-12    -34.00   
         496 2022-09-12    -26.00   
         497 2022-09-09    -48.00   
         498 2022-09-09     -5.69   
         499 2022-09-08     -3.95   
         500 2022-09-08     -1.44   
         501 2022-09-07    -65.95   
         502 2022-09-06   -100.00   
         503 2022-09-06    -61.00   
         504 2022-09-06    -22.59   
         505 2022-09-06    -12.60   
         506 2022-09-06     -4.90   
         507 2022-09-05   -100.00   
         508 2022-09-05    -23.00   
         509 2022-09-01  -2450.50   
         510 2022-09-01    -48.00   
         511 2022-09-01    -48.00   
         512 2022-08-31 -20000.00   
         513 2022-08-30 -20000.00   
         514 2022-08-30    -39.00   
         515 2022-08-30     -9.30   
         516 2022-08-30   3983.32   
         517 2022-08-29    -39.99   
         518 2022-08-29     -7.00   
         519 2022-08-26 -25000.00   
         520 2022-08-25    -19.99   
         522 2022-08-24    -38.00   
         523 2022-08-24     -3.80   
         525 2022-08-22  -1000.00   
         526 2022-08-22  -1000.00   
         527 2022-08-22    -90.00   
         528 2022-08-22    -56.58   
         529 2022-08-22    -48.00   
         531 2022-08-19    150.00   
         532 2022-08-18     -2.44   
         533 2022-08-16    -17.80   
         534 2022-08-15    -55.08   
         535 2022-08-15    -48.00   
         536 2022-08-15    -29.05   
         539 2022-08-08   -116.01   
         540 2022-08-08    -61.00   

                                                           desc  account_nr  \
dkb_3109 477  Kartenzahlung/-abrechnung VIVA RESORT PROT.PRI...  1002423109   
         478  Kartenzahlung/-abrechnung H.RESORT SUPERMERCAD...  1002423109   
         479  Lastschrift Vodafone Deutschland GmbH 09/2022 ...  1002423109   
         480  Lohn, Gehalt, Rente IAV GmbH Ingenieurgesellsc...  1002423109   
         481  Überweisung GISELA SILVA Rechnungs-Nr.: 142/20...  1002423109   
         483  Kartenzahlung/-abrechnung EDEKA GRACKA, BERLIN...  1002423109   
         484  Lastschrift Vodafone GmbH 0000114111198 001835...  1002423109   
         485  Überweisung GISELA SILVA Rechnungs-Nr.: 138/20...  1002423109   
         486  Lastschrift Sauren SICAV SPARPLAN ISIN LU01155...  1002423109   
         487  Lastschrift Sauren SICAV SPARPLAN ISIN LU17170...  1002423109   
         488  Dauerauftrag GIRASOLES-FOERDERVEREIN E. V.  I....  1002423109   
         489  Überweisung GISELA SILVA Rechnungs-Nr.: 133/20...  1002423109   
         490  Kartenzahlung/-abrechnung ALDI SAGT DANKE 28 0...  1002423109   
         491  Kartenzahlung/-abrechnung ASIL GMBH//BERLIN/DE...  1002423109   
         492  Kartenzahlung/-abrechnung Atlantis-Apotheke//B...  1002423109   
         493  Kartenzahlung/-abrechnung BONG APOTHEKE//BERLI...  1002423109   
         495  Kartenzahlung/-abrechnung VINOTRIA GMBH CO KG/...  1002423109   
         496  Kartenzahlung/-abrechnung BEZIRKSAMT CHARLOTTE...  1002423109   
         497  Überweisung GISELA SILVA Rechnungs-Nr.: 129/20...  1002423109   
         498  Kartenzahlung/-abrechnung Burger King 15800//N...  1002423109   
         499  Kartenzahlung/-abrechnung Der Beck GmbH Filial...  1002423109   
         500  Kartenzahlung/-abrechnung MUELLER SAGT DANKE//...  1002423109   
         501  Kartenzahlung/-abrechnung DECATHLON Deuts

### assign target account

In [16]:
r'PAYPAL (EUROPE) S.A.R.L .*'

'PAYPAL (EUROPE) S.A.R.L .*'

In [17]:
df = bc.data_unlabeled

In [18]:
# df[df['desc'].str.match(r'^Gutschrift J.*und C.*')]

In [19]:
# bc.data_unlabeled[bc.data_unlabeled['amount']> 2000]

In [20]:
df_update = assign_target_account(bc.data_unlabeled)
df_update

DEBUG:root:assigning 5 entries to dkb_4217


>>>>PAYPAL (EUROPE) S.A.R.L .*
>>>>^Überweisung HERR .*
>>>>Umbuchung KREDITKARTEN GELDANLAGE
>>>>^Gutschrift J.*und C.*


date   amount  \
dkb_3109 477 2022-09-30  -315.64   
         478 2022-09-30    -4.00   
         479 2022-09-29   -39.99   
         480 2022-09-28  4157.32   
         481 2022-09-27   -69.00   
...                 ...      ...   
         534 2022-08-15   -55.08   
         535 2022-08-15   -48.00   
         536 2022-08-15   -29.05   
         539 2022-08-08  -116.01   
         540 2022-08-08   -61.00   

                                                           desc  account_nr  \
dkb_3109 477  Kartenzahlung/-abrechnung VIVA RESORT PROT.PRI...  1002423109   
         478  Kartenzahlung/-abrechnung H.RESORT SUPERMERCAD...  1002423109   
         479  Lastschrift Vodafone Deutschland GmbH 09/2022 ...  1002423109   
         480  Lohn, Gehalt, Rente IAV GmbH Ingenieurgesellsc...  1002423109   
         481  Überweisung GISELA SILVA Rechnungs-Nr.: 142/20...  1002423109   
...                                                         ...         ...   
         534  Lastschrift Rundfunk ARD, ZDF, DRadio Rundfunk...  1002423109   
         535  Überweisung GISELA SILVA Rechnungs-Nr.: 100/20...  1002423109   
         536  Kartenzahlung/-abrechnung ROSSMANN VIELEN DANK...  1002423109   
         539  Kartenzahlung/-abrechnung AMBROSINO SUPERMARKT...  1002423109   
         540  Lastschrift VATTENFALL EUROPE SALES S/83663436...  1002423109   

             target account class  
dkb_3109 477            NaN   NaN  
         478            NaN   NaN  
         479            NaN   NaN  
         480            NaN   NaN  
         481            NaN   NaN  
...                     ...   ...  
         534            NaN   NaN  
         535            NaN   NaN  
         536            NaN   NaN  
         539            NaN   NaN  
         540            NaN   NaN  

[62 rows x 6 columns]

In [21]:
df_update[~df_update['target account'].isna()]

date   amount  \
dkb_3109 482 2022-09-27   5000.0   
         494 2022-09-19   3000.0   
         521 2022-08-25  20000.0   
         524 2022-08-23  20000.0   
         530 2022-08-22  20000.0   

                                                           desc  account_nr  \
dkb_3109 482  Gutschrift Jan Gieseler und Camila Lobo Costa ...  1002423109   
         494  Gutschrift Jan Gieseler und Camila Lobo Costa ...  1002423109   
         521  Gutschrift Jan Gieseler und Camila Lobo Costa ...  1002423109   
         524  Gutschrift Jan Gieseler und Camila Lobo Costa ...  1002423109   
         530  Gutschrift Jan Gieseler und Camila Lobo Costa ...  1002423109   

             target account class  
dkb_3109 482       dkb_4217   NaN  
         494       dkb_4217   NaN  
         521       dkb_4217   NaN  
         524       dkb_4217   NaN  
         530       dkb_4217   NaN

In [22]:
# df_update.loc['dkb_3109']

In [23]:
bc.data.keys()

dict_keys(['dkb_3109', 'dkb_4217'])

In [24]:
bc.update_data(df_update)

DEBUG:root:updating dataset_name: dkb_3109


ffff Index(['date', 'amount', 'desc', 'account_nr', 'target account', 'class'], dtype='object')


In [29]:
len(bc.data_unlabeled)

57

In [30]:
bc.save_data()

### label that data with regex

In [41]:
bc.data_unlabeled['desc'].value_counts()

Paddle.com Market Limited  500k monthly pageviews  500k monthly pageviews                                                                                           5
Patreon Ireland Limited  Patreon* Membership  Patreon* Membership                                                                                                   5
SOS-Kinderdorf e.V. Germany Donation via PayPal for SOS-Kinderdorf e.V.  Donation via PayPal for SOS-Kinderdorf e.V.                                                3
Paddle.com Market Limited  200k monthly pageviews  200k monthly pageviews                                                                                           3
Tier Mobility GmbH  Temporary hold 7d refund  Temporary hold 7d refund                                                                                              3
                                                                                                                                                                   ..
Tim 

In [42]:
# k = r'^REWE Markt GmbH Germany .* Bestellung .*'
# bc.data_unlabeled[bc.data_unlabeled['desc'].str.lower().str.match(k.lower())]

In [43]:
# bc.data_unlabeled[bc.data_unlabeled['desc'].str.contains('REWE')]

In [44]:
# bc.data_unlabeled

In [45]:
# df_update.loc['paypal', 12]['desc']

In [46]:
df_update = assign_label(bc.data_unlabeled)
print(len(df_update))
df_update

DEBUG:root:labeling 3 entries as 'Home Improvement & Stationary'


150


datetime  amount  \
paypal 0     2019-05-27 07:02:28+00:00 -100.00   
       4     2020-10-06 03:05:58+00:00  -10.98   
       10    2020-10-15 10:24:14+00:00 -127.68   
       16    2020-10-21 20:25:42+00:00  -65.00   
       22    2020-10-29 10:48:06+00:00  -10.00   
...                                ...     ...   
       983   2022-11-23 00:17:17+00:00 -108.94   
       989   2022-11-25 21:03:21+00:00  -52.80   
       999   2022-12-05 13:20:00+00:00 -163.00   
       1009  2022-12-09 00:21:18+00:00  -59.99   
       1011  2022-12-09 10:54:04+00:00  -65.00   

                                                          desc target account  \
paypal 0     Alexander Kihm   Alles Liebe und viel Spass!! ...            NaN   
       4     baby-walz GmbH Germany paypal-payment:62133221...            NaN   
       10    Conrad Electronic SE Germany CE 001500207CE 00...            NaN   
       16    Fatboy the original B.V.  CDE20102195309550  C...            NaN   
       22                       Malte Sagemüller   mittag :-)             NaN   
...                                                        ...            ...   
       983   Nike Retail BV Germany Nike Order  Nike Swoosh...            NaN   
       989       Caudalie SAS Germany 72000462985  72000462985            NaN   
       999   SCC EVENTS GmbH  ORD-0524832. BMW BERLIN-MARAT...            NaN   
       1009                Fashion ID GmbH & Co. KG Germany               NaN   
       1011  CTS EVENTIM Sports GmbH  Tickets Alba Berlin  ...            NaN   

            class  
paypal 0      NaN  
       4      NaN  
       10     NaN  
       16     NaN  
       22     NaN  
...           ...  
       983    NaN  
       989    NaN  
       999    NaN  
       1009   NaN  
       1011   NaN  

[150 rows x 5 columns]

In [47]:
df_update[~df_update['class'].isna()]

datetime  amount  \
paypal 111  2021-01-11 19:38:39+00:00  -74.94   
       509  2021-11-20 11:10:43+00:00  -98.93   
       703  2022-05-02 11:59:11+00:00 -138.96   

                                                         desc target account  \
paypal 111  IKEA Deutschland GmbH & Co. KG Germany IKEA 11...            NaN   
       509  IKEA Deutschland GmbH & Co. KG Germany IKEA 12...            NaN   
       703  IKEA Deutschland GmbH & Co. KG Germany IKEA 12...            NaN   

                                    class  
paypal 111  Home Improvement & Stationary  
       509  Home Improvement & Stationary  
       703  Home Improvement & Stationary

In [48]:
bc.update_data(df_update)

DEBUG:root:updating dataset_name: paypal


In [49]:
bc.save_data()

In [27]:
bc.data_unlabeled

datetime  amount  \
paypal 0     2019-05-27 07:02:28+00:00 -100.00   
       4     2020-10-06 03:05:58+00:00  -10.98   
       10    2020-10-15 10:24:14+00:00 -127.68   
       16    2020-10-21 20:25:42+00:00  -65.00   
       22    2020-10-29 10:48:06+00:00  -10.00   
...                                ...     ...   
       983   2022-11-23 00:17:17+00:00 -108.94   
       989   2022-11-25 21:03:21+00:00  -52.80   
       999   2022-12-05 13:20:00+00:00 -163.00   
       1009  2022-12-09 00:21:18+00:00  -59.99   
       1011  2022-12-09 10:54:04+00:00  -65.00   

                                                          desc target account  \
paypal 0     Alexander Kihm   Alles Liebe und viel Spass!! ...            NaN   
       4     baby-walz GmbH Germany paypal-payment:62133221...            NaN   
       10    Conrad Electronic SE Germany CE 001500207CE 00...            NaN   
       16    Fatboy the original B.V.  CDE20102195309550  C...            NaN   
       22                       Malte Sagemüller   mittag :-)             NaN   
...                                                        ...            ...   
       983   Nike Retail BV Germany Nike Order  Nike Swoosh...            NaN   
       989       Caudalie SAS Germany 72000462985  72000462985            NaN   
       999   SCC EVENTS GmbH  ORD-0524832. BMW BERLIN-MARAT...            NaN   
       1009                Fashion ID GmbH & Co. KG Germany               NaN   
       1011  CTS EVENTIM Sports GmbH  Tickets Alba Berlin  ...            NaN   

            class  
paypal 0      NaN  
       4      NaN  
       10     NaN  
       16     NaN  
       22     NaN  
...           ...  
       983    NaN  
       989    NaN  
       999    NaN  
       1009   NaN  
       1011   NaN  

[151 rows x 5 columns]

In [35]:
bc.data_unlabeled[bc.data_unlabeled['amount']<0]['amount'].sum()

-23162.31

In [32]:
bc.data_unlabeled['amount'].sum()

-23162.31

In [36]:
bc.data_unlabeled.sort_values('amount')

datetime   amount  \
paypal 475  2021-10-24 20:17:25+00:00 -3042.00   
       941  2022-10-18 13:01:12+00:00 -1563.93   
       493  2021-11-14 23:09:30+00:00 -1290.75   
       540  2021-12-24 18:42:43+00:00 -1021.98   
       391  2021-09-06 21:17:41+00:00  -938.00   
...                               ...      ...   
       693  2022-04-27 07:55:12+00:00    -1.00   
       669  2022-04-18 12:59:15+00:00    -1.00   
       833  2022-07-21 05:59:55+00:00    -1.00   
       724  2022-05-12 10:23:45+00:00    -0.90   
       495  2021-11-15 15:43:59+00:00    -0.77   

                                                         desc target account  \
paypal 475                 Bolia International A/S Germany               NaN   
       941  GOL LINHAS AÉREAS S.A  Flight Booking Details ...            NaN   
       493  Airbnb Payments Luxembourg S.A.  Acomodações n...            NaN   
       540  Airbnb Payments Luxembourg S.A.  Acomodações n...            NaN   
       391  Airbnb Payments Luxembourg S.A.  Acomodações n...            NaN   
...                                                       ...            ...   
       693                               Uber Payments BV                NaN   
       669                               Uber Payments BV                NaN   
       833          Shop der Deutschen Post (DPDHL Group)                NaN   
       724  PayPal  Deine Bestellung bei REWE  Deine Beste...            NaN   
       495  Paddle.com Market Limited  500k monthly pagevi...            NaN   

           class  
paypal 475   NaN  
       941   NaN  
       493   NaN  
       540   NaN  
       391   NaN  
...          ...  
       693   NaN  
       669   NaN  
       833   NaN  
       724   NaN  
       495   NaN  

[245 rows x 5 columns]

### define the regex expressions and save to disk

In [40]:
regex_dict = {
    r'^Lastschrift Vodafone .*': 'Bill - Communications',
    r'^Vodafone GmbH .*': 'Bill - Communications',
    r'^Lohn, Gehalt, Rente IAV GmbH .*': 'Income - Salary & Benefits',
    r'^Lastschrift European Bank for Financial Services .*':'Investments',
    r'^Lastschrift Sauren SICAV SPARPLAN .*':'Investments',
    r'^Überweisung GISELA SILVA .*':'Education',
    r'^Dauerauftrag GIRASOLES-FOERDERVEREIN .*':'Education',
    r'^Lastschrift GIRASOLES - SONNENBLUMEN Beitrag .*':'Education',
    r'^buecher.de GmbH & Co. KG*': 'Education',
    r'^Lastschrift VATTENFALL EUROPE .*': 'Bill - Utilities, Rent & Fees',
    r'^Lastschrift Aberdeen Standard Investments .*':'Bill - Utilities, Rent & Fees',
    r'^Kartenzahlung/-abrechnung BARES GbR .*':'Restaurants & Bars',
    r'^app smart GmbH .*':'Restaurants & Bars',
    r'^Takeaway.com Payments B.V.':'Restaurants & Bars',
    r'^Kartenzahlung/-abrechnung EDEKA .*': 'Supermarket & Everyday commodities',
    r'^Kartenzahlung/-abrechnung .*ROSSMANN .*':'Supermarket & Everyday commodities',
    r'^Kartenzahlung/-abrechnung MUELLER.*': 'Supermarket & Everyday commodities',
    r'^Kartenzahlung/-abrechnung ALDI SAGT DANKE .*' : 'Supermarket & Everyday commodities',
    r'^Kartenzahlung/-abrechnung DENN?S BIOMARKT .*' : 'Supermarket & Everyday commodities',
    r'^REWE Markt GmbH Germany .* Bestellung bei REWE .*': 'Supermarket & Everyday commodities',
    r'^DURSTEXPRESS GmbH Germany .*': 'Supermarket & Everyday commodities',
    r'^flaschenpost SE Germany .*': 'Supermarket & Everyday commodities',
    r'.*apotheke.*': 'Personal Care & Sport',
    r'.*Spotify.*': 'Entertainment, Hobbies & Memberships',
    r'^Berliner Bäder-Betriebe Germany.*': 'Entertainment, Hobbies & Memberships',
    r'^LogPay Financial Services GmbH  BVG Tickets .*':'Public Transport',
    r'nextbike GmbH .*':'Public Transport',
    r'DB Vertrieb GmbH  Ihre Buchung bei bahn.de .*':'Public Transport',
    r'^Hansemerkur Krankenversicherung AG.*':'Bill - Insurrance',
    r'^Fashion Retail SA Germany.*': 'Clothes & Shopping',
    r'^H&M Hennes & Mauritz .*': 'Clothes & Shopping',
    r'^Airbnb .*': 'Holidays - Accomodation',
    r'^Ryanair Limited .*': 'Holidays - Transport',
    r'^IKEA Deutschland GmbH & Co. KG .*': 'Home Improvement & Stationary',
}

with open(Path('../data/labeling_regex.yaml'), 'w') as file:
    documents = yaml.dump(regex_dict, file)




In [57]:
with open(Path('../data/labeling_regex.yaml')) as file:
    documents = yaml.full_load(file)

In [58]:
documents

{'.*Spotify.*': 'Entertainment, Hobbies & Memberships',
 '.*apotheke.*': 'Personal Care & Sport',
 '^Dauerauftrag GIRASOLES-FOERDERVEREIN *.': 'Education',
 '^Kartenzahlung/-abrechnung *.ROSSMANN *.': 'Supermarket & Everyday commodities',
 '^Kartenzahlung/-abrechnung ALDI SAGT DANKE *.': 'Supermarket & Everyday commodities',
 '^Kartenzahlung/-abrechnung BARES GbR *.': 'Restaurants & Bars',
 '^Kartenzahlung/-abrechnung DENN?S BIOMARKT *.': 'Supermarket & Everyday commodities',
 '^Kartenzahlung/-abrechnung EDEKA *.': 'Supermarket & Everyday commodities',
 '^Kartenzahlung/-abrechnung MUELLER*.': 'Supermarket & Everyday commodities',
 '^Lastschrift Aberdeen Standard Investments *.': 'Bill - Utilities, Rent & Fees',
 '^Lastschrift European Bank for Financial Services *.': 'Investments',
 '^Lastschrift GIRASOLES - SONNENBLUMEN Beitrag .*': 'Education',
 '^Lastschrift Sauren SICAV SPARPLAN *.': 'Investments',
 '^Lastschrift VATTENFALL EUROPE *.': 'Bill - Utilities, Rent & Fees',
 '^Lastschrif

In [39]:
target_dict = {
    r'PAYPAL (EUROPE) S.A.R.L .*': 'paypal', # filter out the paypal entries
    r'^Überweisung HERR .*': 'dkb',
    r'Umbuchung KREDITKARTEN GELDANLAGE': 'dkb',
    r'^Gutschrift J.*und C.*': 'dkb_4217',
    r'^Überweisung HERR DR.*AN.*': 'dkb_3109'
}
with open(Path('../data/target_account_regex.yaml'), 'w') as file:
    documents = yaml.dump(target_dict, file)

In [40]:
with open(Path('../data/target_account_regex.yaml')) as file:
    documents = yaml.full_load(file)
documents

{'PAYPAL (EUROPE) S.A.R.L .*': 'paypal',
 'Umbuchung KREDITKARTEN GELDANLAGE': 'dkb',
 '^Gutschrift J.*und C.*': 'dkb_4217',
 '^Überweisung HERR .*': 'dkb',
 '^Überweisung HERR DR.*AN.*': 'dkb_3109'}

### manually label data with regex

In [39]:
bc.data_all[bc.data_all['desc'].str.lower().str.match(r'.*IKEA *'.lower())]


datetime  amount  \
paypal   111  2021-01-11 19:38:39+00:00  -74.94   
         485  2021-11-06 21:39:12+00:00 -335.98   
         509  2021-11-20 11:10:43+00:00  -98.93   
         612  2022-03-02 20:16:10+00:00 -453.32   
         703  2022-05-02 11:59:11+00:00 -138.96   
dkb_4217 72         2020-11-16 00:00:00 -128.47   
         391        2021-09-28 00:00:00  -54.18   

                                                           desc  \
paypal   111  IKEA Deutschland GmbH & Co. KG Germany IKEA 11...   
         485  IKEA Deutschland GmbH & Co. KG Germany IKEA 12...   
         509  IKEA Deutschland GmbH & Co. KG Germany IKEA 12...   
         612  IKEA Deutschland GmbH & Co. KG Germany IKEA 12...   
         703  IKEA Deutschland GmbH & Co. KG Germany IKEA 12...   
dkb_4217 72   Kartenzahlung/-abrechnung IKEA 421 BER-TEMPELH...   
         391  Kartenzahlung/-abrechnung IKEA Deutschland Gmb...   

             target account                          class  
paypal   111            NaN                            NaN  
         485            NaN  Home Improvement & Stationary  
         509            NaN                            NaN  
         612            NaN  Home Improvement & Stationary  
         703            NaN                            NaN  
dkb_4217 72             NaN  Home Improvement & Stationary  
         391            NaN  Home Improvement & Stationary

In [32]:
len(bc.data_all)

1324

### manually label

#### label the positive amount as reimbursement

In [18]:
i = bc.data_unlabeled[bc.data_unlabeled['amount']>0].index

In [32]:
bc.data['paypal'].loc[bc.data['paypal']['amount']>0, 'class']= 'Income - Reimbursement'

In [34]:
bc.data['paypal'].loc[bc.data['paypal']['amount']>0, 'class']

1       Income - Reimbursement
3       Income - Reimbursement
5       Income - Reimbursement
7       Income - Reimbursement
9       Income - Reimbursement
                 ...          
1008    Income - Reimbursement
1010    Income - Reimbursement
1012    Income - Reimbursement
1014    Income - Reimbursement
1016    Income - Reimbursement
Name: class, Length: 513, dtype: object

In [24]:
data['paypal'].loc[i, ['class']] = 'Income - Reimbursement'

In [35]:
bc.data_unlabeled

datetime  amount  \
paypal 0     2019-05-27 07:02:28+00:00 -100.00   
       4     2020-10-06 03:05:58+00:00  -10.98   
       6     2020-10-09 17:11:33+00:00  -32.70   
       10    2020-10-15 10:24:14+00:00 -127.68   
       12    2020-10-15 18:46:05+00:00  -10.49   
...                                ...     ...   
       983   2022-11-23 00:17:17+00:00 -108.94   
       989   2022-11-25 21:03:21+00:00  -52.80   
       999   2022-12-05 13:20:00+00:00 -163.00   
       1009  2022-12-09 00:21:18+00:00  -59.99   
       1011  2022-12-09 10:54:04+00:00  -65.00   

                                                          desc target account  \
paypal 0     Alexander Kihm   Alles Liebe und viel Spass!! ...            NaN   
       4     baby-walz GmbH Germany paypal-payment:62133221...            NaN   
       6                        Takeaway.com Payments B.V.                NaN   
       10    Conrad Electronic SE Germany CE 001500207CE 00...            NaN   
       12    buecher.de GmbH & Co. KG Germany Ihre Bestellu...            NaN   
...                                                        ...            ...   
       983   Nike Retail BV Germany Nike Order  Nike Swoosh...            NaN   
       989       Caudalie SAS Germany 72000462985  72000462985            NaN   
       999   SCC EVENTS GmbH  ORD-0524832. BMW BERLIN-MARAT...            NaN   
       1009                Fashion ID GmbH & Co. KG Germany               NaN   
       1011  CTS EVENTIM Sports GmbH  Tickets Alba Berlin  ...            NaN   

            class  
paypal 0      NaN  
       4      NaN  
       6      NaN  
       10     NaN  
       12     NaN  
...           ...  
       983    NaN  
       989    NaN  
       999    NaN  
       1009   NaN  
       1011   NaN  

[331 rows x 5 columns]

In [37]:
bc.save_data()

SyntaxError: unexpected EOF while parsing (23938876.py, line 1)

#### label by string

In [41]:
bc.data_unlabeled[bc.data_unlabeled['desc'].str.contains('Uber Payments BV')]

datetime  amount                  desc  \
paypal 378  2021-08-27 21:15:45+00:00  -19.41  Uber Payments BV       
       555  2022-01-24 09:20:01+00:00   -9.87  Uber Payments BV       
       558  2022-01-29 00:01:45+00:00  -10.68  Uber Payments BV       
       565  2022-02-01 07:15:14+00:00   -2.00  Uber Payments BV       
       566  2022-02-01 07:43:12+00:00  -17.11  Uber Payments BV       
       619  2022-03-10 22:59:41+00:00  -21.52  Uber Payments BV       
       667  2022-04-18 12:59:14+00:00  -15.26  Uber Payments BV       
       669  2022-04-18 12:59:15+00:00   -1.00  Uber Payments BV       
       691  2022-04-27 07:47:07+00:00  -52.01  Uber Payments BV       
       693  2022-04-27 07:55:12+00:00   -1.00  Uber Payments BV       

           target account class  
paypal 378            NaN   NaN  
       555            NaN   NaN  
       558            NaN   NaN  
       565            NaN   NaN  
       566            NaN   NaN  
       619            NaN   NaN  
       667            NaN   NaN  
       669            NaN   NaN  
       691            NaN   NaN  
       693            NaN   NaN

In [42]:
i = bc.data_unlabeled[bc.data_unlabeled['desc'].str.contains('Uber Payments BV')].index

In [48]:
i = bc.data['paypal'][bc.data['paypal']['desc'].str.contains('Uber Payments BV')].index

In [50]:
bc.data['paypal'].loc[i, ['class']] = 'Public Transport'

In [51]:
bc.data['paypal'].loc[i, ['class']]

,class
378,Public Transport
555,Public Transport
558,Public Transport
565,Public Transport
566,Public Transport
619,Public Transport
667,Public Transport
669,Public Transport
691,Public Transport
693,Public Transport


In [52]:
bc.save_data()

### rename and merge labels

In [116]:
old_label = 'Education'
new_label = 'Education & Books'


# old_label = 'Home Improvement & Stationary'
# new_label = 'Home Improvement, Furniture & Stationary'



In [117]:
bc = BankClassify(Path('../data'))

DEBUG:root:FFFFF /Users/jangie/Projects/DataScience/BankClassify/BankClassify.py
INFO:root:loaded previous dkb_3109 data 541 entries
INFO:root:loaded previous paypal data 1017 entries
INFO:root:loaded previous dkb_4217 data 1212 entries
INFO:root:train dataset size 2219


In [118]:
for k, df in bc.data.items():
    print(k)
    print(f"number of old labels: {len(df[df['class'] == old_label])}")
    print(f"number of new labels: {len(df[df['class'] == new_label])}")
    # df[df['class'] == old_label] =new_label
    df.loc[df['class'] == old_label, 'class'] = new_label
    print(f"number of new labels: {len(df[df['class'] == new_label])}")

dkb_3109
number of old labels: 68
number of new labels: 0
number of new labels: 68
paypal
number of old labels: 7
number of new labels: 0
number of new labels: 7
dkb_4217
number of old labels: 2
number of new labels: 0
number of new labels: 2


In [119]:
bc.save_data()

In [70]:
len(df[df['class'] == new_label])

48